# Python for Harvesting Data on the Web

**Nicholas Wolf and Vicky Steeves, NYU Data Services**

Vicky's ORCID: 0000-0003-4298-168X | Nick's ORCID: 0000-0001-5512-6151

This work is licensed under a Creative Commons Attribution-NonCommercial 4.0 International License.

## Overview

This session is an intermediate-to-advanced level class that offers some ideas for how to approach the following common data wrangling needs in research:

 - Obtain data and load it into a suitable data "container" for analysis, often via a web interface, especially an API
 - Parse the data retrieved via an API and turn it into a useful object for manipulation and analysis
 - Perform some basic summary counts of records in a dataset and work up a quick visualization

## Setup

**Project Environment**

Download the notebook available at [https://goo.gl/Pnm7Dx](https://goo.gl/Pnm7Dx) and open it in Jupyter Notebook. Alternatively, you can clonse the course materials using

<code>git clone https://github.com/NYU-DataServices/DataHarvesting-Python.git</code>

**State of New York Socrata API Account**

To work with this session's data, you'll need to create an API account with the state of New York's data service. Visit <a href="https://data.ny.gov">data.ny.gov</a> and click on signup to create an account.

# 1. Web Requests

It is not unusual to find open data served directly on the web, without requiring any tokens or authorizations. Although it is not often the practice to provide very large resources via such an entrypoint because of the burden it places on data servers, smaller datasets are frequently available. When it comes to such non-API generated data it is generally easier to work with services that serve the data directly in its native file format, rather than wrapping it in HTML. The latter requires you to parse first the HTML or interpret the HTTP response.

As an example, take the motor vehicle accident report <a href="https://data.ny.gov/Transportation/Motor-Vehicle-Crashes-Vehicle-Information-Three-Ye/xe9x-a24f">data found here</a>. One thousand records from this 1.65 million-record dataset can be accessed directly at <a href="https://data.ny.gov/resource/cm56-widp.json">https://data.ny.gov/resource/cm56-widp.json</a> (FYI Firefox has a nice JSON viewer built in when you encounter .json hosted files on the web.)

Let's walk through making an HTTP request for that .json data and quickly transforming it into a useful container (a Pandas dataframe) to ready it for use. We'll make use of Python's <a href="https://docs.python.org/3/library/json.html">JSON module</a>, a compact and easy-to-use way of turning JSON into Python's native object types, lists and dictionaries. If you want to anticipate what JSON input will prompt which Python object type output, see this table here: https://docs.python.org/3/library/json.html#json-to-py-table. Note that a JSON array of key-value objects will yield a Python list of dictionaries.


In [ ]:
import json
import requests

mv_data_json = requests.get('https://data.ny.gov/resource/cm56-widp.json')

mv_list_recs = json.loads(mv_data_json.text)

print(mv_list_recs[0])

**Pandas**

If you haven't yet given a Pandas dataframe a try as a way to manage large arrays of information, give it a go. It <em>is</em> possible to go overboard: not everything needs to be put in a dataframe, especially when a Python list of lists or a dictionary will do. Take a look at how fast we can access subsets of the motor vehicle data.

In [ ]:
import pandas as pd
from IPython.display import display

mv_df = pd.DataFrame(mv_list_recs)

display(mv_df.head(10))

In [ ]:
print(type(mv_df.number_of_occupants))
print(type(mv_df.number_of_occupants.tolist()))

** Bonus: wget direct from web**

If you have wget installed on your system, you can use the command line utility wget directly in a Notebook cell.

To install wget, visit [http://www.gnu.org/software/wget/](http://www.gnu.org/software/wget/)

In [ ]:
!wget https://www.nhc.noaa.gov/data/hurdat/hurdat2-nepac-1949-2017-050418.txt

In [ ]:
with open('hurdat2-nepac-1949-2017-050418.txt') as f:
    for i in f.readlines()[0:2]:
        print(i)

# Web API Requests

For more robust ways of serving up data, i.e. by APIs, we generally need to register an "app," i.e. an application that will be accessing the data, receive at the very least a token (and often a client secret as well) to enable tracked downloads of data, ensure proper access limits, etc.

It is very helpful if an API comes with a pre-built library to interface with that server so that you don't have to handle signing requests in HTTP, managing tokens, etc. For example, here is the typical workflow often required for authenticated API interfacing:

In [ ]:
import requests
import json
from requests import Request

## This first step is just to authenticate user and receive a token (which may or may not expire) to interface with the API

def token_retrieve():
    authurl = 'https://TOKEN-API-RETRIEVE-URL'
    authparams = {'client_id':'CLIENT-ID',
              'response_type':'code',
              'scope':'/authenticate',
              'redirect_uri':'https://PRE-ESTABLISHED-REDIRECT-URL'
    }
    
    code_token = Request('POST', authurl, params=authparams)
    
    ## In between step above and below some kind of password authentication takes place.
    
    tokenurl = 'https://URL-TO-CONFIRM-TOKEN'
    
    head = {'content-type':'Accept: application/json'}
    params = {'client_id':'CLIENT-ID',
              'client_secret':'CLIENT-SECRET',
              'code':continput,
              'grant_type':'authorization_code',
              'redirect_uri':'https://PRE-ESTABLISHED-REDIRECT-URL'
    }

    r = requests.post(tokenurl, data=params)

    return json.loads(r.text)['access_token']

## Now that we are authenticated we can finally get the data.

def get_data(acc_token):
    baseurl = 'https://DATA-API-URL'
    head = {'Accept': 'application/vnd.orcid+json',
            'Authorization':'Bearer ' + acc_token
    }
    r = requests.get(baseurl + 'API-PARAMETERS', headers=head)
    return r.text

Each of these little mini-apps needs to be configured to work with a given API, and this hasn't even arrived at the question of how best to structure your data responses so that they are easy to work with.

Fortunately, we have a nice workable pre-made library for working with this same NY State Socrata API portal. The module is called <a href="https://github.com/xmunoz/sodapy/blob/master/examples/basic_queries.py"><pre>sodapy</pre></a>

We'll need to install it:

<pre>pip install sodapy</pre>

or

<pre>easy_install sodaypy</pre>

or

<pre>conda install sodapy</pre>

**Web API Access with Prebuilt Library**

Helpfully, this sodapy has a nice means of working specifically with the NY State Socrata platform. Once you have sodapy installed, you'll need to click on the large "Sign up for an app token!" button here: https://dev.socrata.com/foundry/data.ny.gov/cm56-widp. Or you can select create a new application and begin registering your API retriever via your State Gov dashboard.

One you have created an App, copy and paste the App token somewhere for the next step.


In [ ]:
import sodapy
from sodapy import Socrata

client = Socrata('data.ny.gov','TOKEN',
                username='EMAIL-USERNAME',
                password='PASSWORD')

results = client.get('cm56-widp', limit=200000)

In [ ]:
results_df = pd.DataFrame.from_records(results)
display(results_df.head(10))

## Challenge

What if we need to know if a column's values are unique? In the motor vehicle dataframe, for example, what if we suspect that case_vehicle_id is nonunique but should be? How can we check it?

**Quick Frequency Counts on a Subset of Data**

If we are interested in, say, the counts of each state to which a vehicle involved in an incident was registered, we can do this quickly in Pandas using a groupby function and summing up counts of those grouped common values. 

Let's also address that NULL value problem:

In [ ]:
num_occs = results_df.dropna(axis=0, how='any', 
                             subset=['number_of_occupants'])

num_occs.number_of_occupants.value_counts()

We can do the same to discover typical numbers of occupants in vehicles:


In [ ]:
layer_df = results_df.dropna(axis=0, how='any',
                            subset=['action_prior_to_accident',
                                   'number_of_occupants']).groupby(['action_prior_to_accident',
                                                                  'number_of_occupants']).size()

display(layer_df)

## Easy Plots

Finally, let's look at putting this all together with the quick matplot visualization capabilities that work really well in Jupyter Notebook and even integrate nicely with Pandas.

Let's work with the water use data provided by NYC's open data portal. Now we are working on the city level and not the state: <a href="https://data.cityofnewyork.us/Environment/Water-Consumption-In-The-New-York-City/ia2d-e54m">Water consumption data from the NYC Open Data portal</a>. Preview the JSON version <a href="https://data.cityofnewyork.us/api/views/ia2d-e54m/rows.json?accessType=DOWNLOAD">here</a>.

Our workflow is the same: pull the data in JSON format from the web download, turn it into a dataframe, and then visualize.

Note the plotting library we pull in, plus a means (via a magic command) to display the visualization in-line.

In [ ]:
import requests
import matplotlib.pyplot as plt

%matplotlib inline

water_data = requests.get('https://data.cityofnewyork.us/api/views/ia2d-e54m/rows.json?accessType=DOWNLOAD')

water_dictionary = json.loads(water_data.text)

full_data = []

for row in water_dictionary['data']:
    full_data.append(row)
    
col_names = [col_info['name'] for col_info in water_dictionary['meta']['view']['columns']]

water_df = pd.DataFrame(full_data, columns = col_names)

display(water_df)

In [ ]:
water_df[['NYC Consumption(Million gallons per day)','Per Capita(Gallons per person per day)']] = \
    water_df[['NYC Consumption(Million gallons per day)','Per Capita(Gallons per person per day)']].astype(float)

    
#ax = water_df.plot(x='Year', y='Per Capita(Gallons per person per day)', 
              #figsize=(10,10), title='Declining Water Use Over Time')
#ax.set_xlabel('Year')
#ax.set_ylabel('Per Capita Consumption in Gallons')
#plt.show()

water_df.plot(x='Year', y='Per Capita(Gallons per person per day)', figsize=(10,10), title='Declining Water Use Over Time')

## Challenge
 
Try making a new plot that only displays the years 1990 onward.
 